# Project and Data Description

#### AIM: 
Set up a database to store data of 10 mutual funds and optimize the format/schema to store this data. Build a query to calculate 1/2/3/7- and 10-year annualized return of each mutual fund and store that in the database. Finally, showcase the same using a UI form.

This project can be scaled to automatically update databases on a daily basis based on new market data and host the UI in an accessible manner (publicly or just privately within a firm) to use the returns value for fund analysis. It can also be scaled to include other fund metrics and ultimately, build a fund analysis tool.

#### DATA:
This project is designed to take data as a user input. However, just for testing, I have used the following ten funds to build my database and UI.

- JPMorgan Equity Income Fund (OIEIX)
- JPMorgan Large Cap Growth Fund (JLGRX)
- JPMorgan Growth Advantage Fund (JGASX)
- Fidelity Blue Chip Growth Fund (FBGRX)
- PIMCO Emerging Markets Bond Fund (PEMPX)
- Goldman Sachs Large Cap Core Fund (GSCGX)
- Goldman Sachs Mid Cap Growth Fund (GGOIX)
- Vanguard Equity Income Fund (VEIPX)
- Goldman Sachs Growth and Income Strategy Portfolio (GPIRX)
- Vanguard Value Index Fund (VIVIX)

The data being used for each fund is the daily Adjusted Close price. I have used Adjused Close as it represents the closing price after adjusting for splits and dividends. Thus, it gives a more complete picture compared to Closing Price.

# 0.) Import Required Packages

In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import pymysql
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from flask import Flask
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import plotly.express as px

# 1.) Database Creation

This function creates a new database on your local MySQL connection to store the data of ten selected mutual funds.

##### It requires the following inputs:
- A list of ticker values for your choice of mutual funds.
- Password for your localhost SQL connection.

##### The function goes through the following steps:
1. Pulls data from Yahoo Finance for the selected tickers and converts them into monthly data.
2. Creates a new database in SQL.
3. Creates a new table to store mutual fund data.
4. Inserts data from Yahoo Finance into the table in a long format.

##### Format Optimization:
- The mutual funds data is stored in a single long format table to optimize storage.
- I tried creating 10 different tables, one for each fund. However, such a schema would make it hard to retrieve data for all funds at the same time for the same date. It would also not be dynamic and each fund addition would require the creation of a new table. This would not be optimal in the case of say, 100 funds!

_Why long format?_
- This format is more dynamic when considering updating with new data as it just appends such data. For ex: new funds or new dates for same funds.
- This format is faster to load and requires less storage since creates only one table instead of 10.
- It is also easy to access data by using 'where' clause on a selected ticker.




In [376]:
# Select your fund tickers
tickers = ["OIEIX","JLGRX","JGASX","FBGRX", "PEMPX", "GSCGX", "GGOIX", "VEIPX", "GPIRX", "VIVIX"]

# Enter password for your local MySQL connection
#password = '###'

In [375]:
def create_db(tickers):
    #creating data dictionary to pull data from yahoo finance
    data = {}
    for t in tickers:
        daily = yf.download(t)[["Adj Close"]]
        #converting to monthly data
        data[f'fund{t}'] = daily.resample('M').mean()
    
    #establishing sql connection
    endpoint = 'localhost'
    username = 'root'
    connection = pymysql.connections.Connection(host=endpoint,
                                                user=username,
                                                password=password)

    #creating new database in local connection
    connection.cursor().execute("CREATE DATABASE IF NOT EXISTS Funds")
    connection.commit()

    connection.cursor().execute("USE Funds")
    connection.commit()

    #creating a long format table with all funds' data
    tbname = "funds"
    table_query = "CREATE TABLE IF NOT EXISTS {} (Date date, Adj_Close FLOAT, Ticker VARCHAR(255))".format(tbname)
    connection.cursor().execute(table_query)
    connection.commit()

    for fund in data:
        unindexed = data[fund].reset_index()
        ticker_val = str(fund)
        for i in range(len(data[fund])):
            sql = "INSERT INTO {} (Date, Adj_Close, Ticker) VALUES (%s,%s,%s)".format(tbname)
            date_value = unindexed.iloc[i, 0].strftime('%Y-%m-%d')
            connection.cursor().execute(sql, (date_value, unindexed.iloc[i,1], ticker_val))
            connection.commit()

In [377]:
# Execution
create_db(tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Check your SQL database and explore the data!

# 2.) Annualized Returns Summary

#### Annualized Return calculator
Defining a function to take in user defined dataframe containing fund NAVs and user defined time period to calculate annualized returns.

In [378]:
def annualized(fund, year):
    end_date = fund.index[-1].strftime("%Y-%m-%d")
    start_date = (fund.index[-1] - relativedelta(years=year)).strftime("%Y-%m-%d")
    
    start_val = fund.loc[start_date:end_date].iloc[0,0]
    end_val = fund.loc[start_date:end_date].iloc[-1,0]
    
    return ((end_val/start_val)**(1/year) - 1 )*100

#### Query to store returns summary in the database
This function queries data from previously created funds table in SQL, calculates annualized returns for given time periods and store the same back in the database as a new table.

In [380]:
def create_returns():
    connection.cursor().execute("USE funds")
    connection.commit()
    
    #creating new table to store annualized returns for all funds
    connection.cursor().execute("CREATE TABLE IF NOT EXISTS returns (Fund varchar(255), 1yr FLOAT, 2yr FLOAT, 3yr FLOAT, 7yr FLOAT, 10yr FLOAT)")
    connection.commit()
    
    for t in tickers:
        fund = f'fund{t}'
        cursor = connection.cursor()
        select = "SELECT * FROM funds WHERE Ticker =(%s)"   #slicing table
        cursor.execute(select,fund)
        result = cursor.fetchall()
        
        #converting sql output into table for calculations
        data2 = pd.DataFrame(result, columns=['Date', 'Adj_Close', 'Ticker'])
        data2['Date'] = pd.to_datetime(data2['Date'])
        data2.set_index('Date', inplace=True)
        
        #returns calculations
        yr1 = annualized(data2, 1)
        yr2 = annualized(data2, 2)
        yr3 = annualized(data2, 3)
        yr7 = annualized(data2, 7)
        yr10 = annualized(data2, 10)
    
        #storing into database
        sql = "INSERT INTO returns (Fund, 1yr,2yr,3yr,7yr,10yr) VALUES (%s,%s,%s,%s,%s,%s)"
        connection.cursor().execute(sql, (fund, yr1, yr2, yr3, yr7, yr10))
        connection.commit()

In [381]:
# Execution
create_returns()

# 3.) UI Creation

##### This function does the following:
1. Calls returns table from local database into Python for UI creation.
2. Formats SQL table into pandas dataframe.
3. Builds an App based on Dash to display the returns table and build an interactive chart using plotly to allow the user to select different funds and view their return distributions. This app has a checkbox feature which allows multiple selections simultaneously, updating the chart in real time.
    - You can refer to the attached video/gif to understand how it works.
    - There is also an executable python script in this repository which allows to explore the live dashboard (please download the 'returns' CSV file before execution).
4. Finally, hosts the UI display locally using Flask.

In [382]:
def create_dashboard():
    connection.cursor().execute("USE funds")
    connection.commit()
    
    #calling returns table from sql
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM returns")
    result = cursor.fetchall()
    
    #converting to pandas format
    returns = pd.DataFrame(result, columns=['Fund', '1yr','2yr','3yr','7yr','10yr'])
    returns_trial = returns.set_index('Fund')
    returns_transposed = returns_trial.T
    df1 = returns_transposed.reset_index()
    df1 = df1.rename(columns={'index': 'period'}).rename_axis(None, axis=1)
    
    #for checkbox display
    tickers_ = [df1.columns[1:][i][4:] for i in range(len(df1.columns[1:]))]
    funds = df1.columns[1:]
    options = [{"label": tickers_[i], "value": funds[i]} for i, _ in enumerate(funds)]

    # Initialize the app
    app = Dash(__name__)

    # App layout
    app.layout = html.Div([
        html.H1('Mutual Funds - Annualized Returns'),
        html.Hr(),
        dcc.Checklist(options=options,inline=True, value=['fundOIEIX'],id='checklist'),
        #inline=True for horizontal checkboxes
        #'value' is default selection
        html.Br(),
        dash_table.DataTable(data=df1.to_dict('records')),
        dcc.Graph(figure={}, id='controls-and-graph')     
            #id names are used for callback and inetraction
    ])     #figure here is empty because it'll display based on user selection/default

    # Add controls to build the interaction
    @callback(
        Output(component_id='controls-and-graph', component_property='figure'),
        Input(component_id='checklist', component_property='value')
    )
    def update_graph(col_chosen):
        colors = px.colors.sequential.Blues
        fig = px.histogram(df1, x='period', y=col_chosen, histfunc='avg',
                           color_discrete_sequence=colors[8:3:-1])
        fig.update_layout(
        barmode='group',
        bargroupgap=0.2,
        xaxis=dict(title='Period'),
        yaxis=dict(title='Annualized Returns'),
        legend_title_text='Funds'
    )
        return fig

    # Run the app
    if __name__ == '__main__':
        app.run_server()

In [383]:
# Execution
create_dashboard()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


# 4.) Appendix

This section has code that was tried and not used for final execution. It also contains a few generic outputs generated during project building and is displayed here to understand what the data looks like.

In [7]:
## Shows how the data from Yahoo Finance looks

fund_1 = yf.download("OIEIX")[["Adj Close"]]
fund_1

[*********************100%***********************]  1 of 1 completed


,Adj Close
Date,
1992-02-18 00:00:00-05:00,1.657202
1992-02-19 00:00:00-05:00,1.657202
1992-02-20 00:00:00-05:00,1.657202
1992-02-21 00:00:00-05:00,1.657202
1992-02-24 00:00:00-05:00,1.657202
...,...
2023-04-19 00:00:00-04:00,21.920000
2023-04-20 00:00:00-04:00,21.799999
2023-04-21 00:00:00-04:00,21.790001


In [66]:
## Shows the value annualized function returns

annualized(fund_1,1)

-2.5449547896996716

In [21]:
## How the data for each fund looks within the data dictionary created

data["fundJLGRX"]

,Adj Close
Date,
2009-04-14 00:00:00-04:00,5.704672
2009-04-15 00:00:00-04:00,5.718396
2009-04-16 00:00:00-04:00,5.828188
2009-04-17 00:00:00-04:00,5.832764
2009-04-20 00:00:00-04:00,5.626901
...,...
2023-04-21 00:00:00-04:00,50.290001
2023-04-24 00:00:00-04:00,50.160000
2023-04-25 00:00:00-04:00,49.070000


In [85]:
## The loop that created different tables for each mutual fund in the SQL database -- scrapped!

for fund in data:
    unindexed = data[fund].reset_index()
    tbname = fund
    table_query = "CREATE TABLE IF NOT EXISTS {} (Date date PRIMARY KEY, Adj_Close FLOAT)".format(tbname)
    connection.cursor().execute(table_query)
    connection.commit()
    for i in range(len(data[fund])):
        sql = "INSERT INTO {} (Date, Adj_Close) VALUES (%s,%s)".format(tbname)
        date_value = unindexed.iloc[i, 0].strftime('%Y-%m-%d')
        connection.cursor().execute(sql, (date_value, unindexed.iloc[i,1]))
        connection.commit()

In [123]:
## Shows what data from SQL looks like -- and why it needs to be converted for use in python

cursor = connection.cursor()
cursor.execute("SELECT * FROM returns")
result = cursor.fetchall()

for x in result:
    print(x)

('fundOIEIX', -1.38116, 2.38787, 13.9658, 9.51689, 9.7462)
('fundJLGRX', -0.273622, -5.52802, 13.3733, 17.2703, 15.4352)
('fundJGASX', -3.22, -6.96265, 13.1889, 15.6401, 15.4117)
('fundFBGRX', -2.84775, -9.80218, 13.7036, 16.0034, 15.4259)
('fundPEMPX', -2.7404, -7.56515, 0.412597, 1.95308, 1.1807)
('fundGSCGX', 1.82758, -0.958647, 13.8049, 12.6777, 12.3142)
('fundGGOIX', -1.15703, -10.9606, 8.80132, 10.4123, 9.93794)
('fundVEIPX', 0.179058, 4.5908, 14.5077, 9.83061, 10.0803)
('fundGPIRX', -1.33814, -3.20388, 6.81077, 5.3373, 4.6077)
('fundVIVIX', -0.507799, 3.6657, 14.9412, 10.0613, 10.3186)


In [192]:
type(result)

tuple

In [129]:
result[0]

('fundOIEIX', -1.38116, 2.38787, 13.9658, 9.51689, 9.7462)

In [131]:
## Creating pandas df of the same
returns = pd.DataFrame(result, columns=['Fund', '1yr','2yr','3yr','7yr','10yr'])

In [132]:
returns

,Fund,1yr,2yr,3yr,7yr,10yr
0,fundOIEIX,-1.381160,2.387870,13.965800,9.51689,9.74620
1,fundJLGRX,-0.273622,-5.528020,13.373300,17.27030,15.43520
2,fundJGASX,-3.220000,-6.962650,13.188900,15.64010,15.41170
3,fundFBGRX,-2.847750,-9.802180,13.703600,16.00340,15.42590
4,fundPEMPX,-2.740400,-7.565150,0.412597,1.95308,1.18070
5,fundGSCGX,1.827580,-0.958647,13.804900,12.67770,12.31420
6,fundGGOIX,-1.157030,-10.960600,8.801320,10.41230,9.93794
7,fundVEIPX,0.179058,4.590800,14.507700,9.83061,10.08030
8,fundGPIRX,-1.338140,-3.203880,6.810770,5.33730,4.60770
9,fundVIVIX,-0.507799,3.665700,14.941200,10.06130,10.31860


In [310]:
## Exporting returns table as CSV for DashboardExecutable file

returns.to_csv('C:/Users/agarw/Desktop/returns.csv')